# S2_Get_Geo_locator_from_address.ipynb
## Descriptions
* Address as string
* Convert to geo locator using osm 

## Pool the address

In [1]:
import sqlite3 as lite
def select_addr(conn):
    """
    """
    cur = conn.cursor()
    cur.execute("select ci_text from corp_info where ci_name in ('REGISTERED OFFICE ADDRESS','DIRECTORS')") 
    rows = cur.fetchall()
    return([r[0] for r in rows])

ddb = "./data/corporate/corp.db"                    

con = lite.connect(ddb)
l_addr = list(set(select_addr(con)))
l_addr = l_addr[1:]
con.close()

## Pre-process Address 

In [2]:
# extract the pretext before the address

import re
def getPreText(txtaddr):
    preText = " : "
    m = re.search("([^0-9]*:[^0-9]*)*", txtaddr)     
    strRemain = txtaddr
    try:
        strRemain = str.replace(txtaddr,str(m.group(0)),"")
    except Exception:
        pass
    try:
        preText = str(m.group(0)).strip()
    except Exception:
        pass
    
    field = ''
    txt = ''
    try:
        l_buf = preText.split(":")
        field = l_buf[0]
        txt = l_buf[1]
    except:
        pass
    return ((txt.lower().strip(), field.lower().strip()), strRemain)
    
print(getPreText('460, RUE SAINTE-CATHERINE OUEST BUREAU 503 MONTREAL QC H3B 1A7 CANADA'))

(('', ''), '460, RUE SAINTE-CATHERINE OUEST BUREAU 503 MONTREAL QC H3B 1A7 CANADA')


## Level 1: Generate Address Key for Address Identifier

### libpostal: international street address NLP
#### Install libpostal 
https://github.com/openvenues/libpostal
#### Install python package postal
pip install postal

In [112]:
# Function to create standardize addr_key
# try to standardize address 

# https://github.com/openvenues/libpostal
# pip install postal

import re
def splitstr(a):
    b = a.translate ({ord(c): " " for c in "!@#$%^&*()[]{};:,./<>?\|`~-=_+"})
    return b.split()

def rplceToken(srch_pat, rplce_pat, txtaddr):        
    m = re.search(srch_pat, txtaddr) 
    strRplce = txtaddr
    try:
        strRplce = str.replace(txtaddr,str(m.group(0)),rplce_pat)
    except Exception:
        pass
    return (strRplce)

def rplcRoadToken(road):
    strR4= rplceToken("(drive[\s\,])", "dr ", road)
    strR5= rplceToken("(boulevard[\s\,])", "blvd ", strR4)
    strR6= rplceToken("(crescent[\s\,])", "cres ", strR5)
    strR7= rplceToken("(street[\s\,]|STR[\s\,])", "st ", strR6)
    strR8= rplceToken("(road[\s\,]|RAOD[\s\,])", "rd ", strR7)
    strR8A= rplceToken("(avenue[\s\,])", "ave ", strR8)
    cleanedRoad = rplceToken("(highway[\s\,])", "hwy ", strR8A)
    return cleanedRoad

def gen_addr_key(rec):
    dict_addr = dict(zip(([v[1] for v in rec]), ([v[0] for v in rec])))    
    try:
        buf = "_".join([e[:2] for e in splitstr(dict_addr['country'])]) + "|"  
    except:
        buf = "_" + "|"          
    try:
        buf = buf + dict_addr['postcode'].replace(" ","").replace(",","").replace("-","") + "|"  
    except:
        buf = buf + "_" + "|"  
    try:
        buf = buf + "_".join([e[:2] for e in splitstr(dict_addr['state'])]) + "|"   
    except:
        buf = buf + "_" + "|"  
    try:
        buf = buf + "_".join([e[:2] for e in splitstr(dict_addr['city'])]) + "|"    
    except:
        buf = buf + "_" + "|"  
    try:
        buf = buf + "_".join([e[:2] for e in splitstr(rplcRoadToken(dict_addr['road']))]) + "|"    
    except:
        buf = buf + "_" + "|"  
    try:
        buf = buf + re.sub("[^0-9]", "", dict_addr['house_number']) + "|" 
    except:
        buf = buf + "_" + "|" 
    try:
        buf = buf + re.sub("[^0-9]", "", dict_addr['unit']) + "|" 
    except:
        buf = buf + "_" + "|"         
    try:
        buf = buf + re.sub("[^0-9]", "", dict_addr['po_box']) + "|" 
    except:
        buf = buf + "_" + "|" 

    return buf

In [110]:
# Test Case 1
_addr = '640 BOULEVARD ROBERVAL OUEST LONGUEUIL QC J4L 3B7 CANADA'
rec = parse_address(_addr)
dict_addr = dict(zip(([v[1] for v in rec]), ([v[0] for v in rec]))) 
print(dict_addr['road'])
rplcRoadToken(dict_addr['road'])

boulevard roberval ouest


'blvd roberval ouest'

In [91]:
# Test Case 2
_addr = "6 PORTNEUF C.P. 399 CANTLEY QC J0X 1L0 CANADA"
parse_address(_addr)

[('6', 'house_number'),
 ('portneuf', 'road'),
 ('c.p. 399', 'po_box'),
 ('cantley', 'city'),
 ('qc', 'state'),
 ('j0x 1l0', 'postcode'),
 ('canada', 'country')]

In [116]:
# Test Case 3
from postal.parser import parse_address
for addr in l_addr[0:10]:
    t, _addr = getPreText(addr)
    rec = parse_address(_addr)
    if len(t[0]) > 0:
        rec.append(t) 
    print("ADDR:   " + addr)
    print("KEY:    " + gen_addr_key(rec))

ADDR:   673 RUE GILLES RIMOUSKI QC G5L 3L4 CANADA
KEY:    ca|g5l3l4|qc|ri|ru_gi|673|_|_|
ADDR:   CHEMIN LAMARCHE ANGERS QC J0X 1B0 CANADA
KEY:    ca|j0x1b0|qc|an|ch_la|_|_|_|
ADDR:   49 LOUIS ST-LAURENT AYLMER QC J9H 5X9 CANADA
KEY:    ca|j9h5x9|qc|ay|lo_st_la|49|_|_|
ADDR:   5770 ROYALMOUNT MONTREAL QC H4P 1K5 CANADA
KEY:    ca|h4p1k5|qc|mo|ro|5770|_|_|
ADDR:   8820, CROISSANT DU LOUVRE APP. #2 BROSSARD QC J4Y 0C6 CANADA
KEY:    ca|j4y0c6|qc|br|cr_du_lo|8820|2|_|
ADDR:   28TH FLOOR 25 YORK ST. TORONTO ON M5J 2V5 CANADA
KEY:    ca|m5j2v5|on|to|yo_st|25|_|_|
ADDR:   204-512, RIVERSIDE DRIVE LONDON ON N6H 5E2 CANADA
KEY:    ca|n6h5e2|on|lo|ri_dr|204512|_|_|
ADDR:   2039 ARDLEIGH ROAD OAKVILLE ON L6J 1V6 CANADA
KEY:    ca|l6j1v6|on|oa|ar_ro|2039|_|_|
ADDR:   34 MAPLE SUGAR LANE VAUGHAN ON L4J 8S5 CANADA
KEY:    ca|l4j8s5|on|va|ma_su_la|34|_|_|
ADDR:   1 NEWPORT DR SHERWOOD PARK AB T8A 2A1 CANADA
KEY:    ca|t8a2a1|ab|sh_pa|ne_dr|1|_|_|


## Output files for Graph Generation
* Tables to input to neo4j   
#### address_nodes.csv  
* addr_key: primary key, varchar  
* addr_addr: full address, varchar  
* addr_long: longitude, float  
* addr_lat: latitude, float  
#### director_nodes.csv  
* 


# (not cleaned, ignore below for now)
## LEVEL 2: Generate Geo-locator for Address Identifier 

### Postal Code
* Forward sortation areas (FSA)  
NL        -                 A  
NS        -                 B  
PE        -                 C  
NB        -                 E  
QC        -                 G, H, J  
ON        -                 K, L, M, N, P  
MB        -                 R  
SK        -                 S  
AB        -                 T  
BC        -                 V  
NU/NT     -                 X  
YT        -                 Y  
* Local delivery units (LDU)  
LDUs ending in zero correspond to postal facilities, from post offices and small franchised retail postal outlets all the way up to sortation plants. 

In [ ]:
import re
addr = "j8p 1r4"
m = re.search('([A-Za-z]\d[A-Za-z]) ?(\d[A-Za-z]\d)', addr)
print (m.group(1))

In [ ]:
import re
def getPostalCode(addr):
    m = re.search('([A-Za-z]\d[A-Za-z]) ?(\d[A-Za-z]\d)', addr)
    return m.group(0), m.group(1), m.group(2)

for addr in l_addr[:10]:
    try:
        PC, FSA, LDU = getPostalCode(addr)
    except:
        PC = "UNK"
        FSA = "UNK"
        LDU = "UNK"
    print(addr + ": " + PC + "," + FSA + "," + LDU)

In [ ]:
def getStrInParentheses(Instr):
    m = re.findall("\((.*?)\)", Instr)
    if m == []:
        m = ""
        n = Instr
    else:
        m = m[len(m)-1]
        n = (Instr[:-1*(len(m)+3)])
    l = n.split(" ")
    return (l, m)

print(getStrInParentheses("STEVE ODENY (SECRETARY)"))

### 
#### CREATE TABLE corp_addr_ext(
####     ci_corp_id INT,
####     ci_name TEXT,
####     ci_lab TEXT,
####     ci_text TEXT, ci_postal TEXT, ci_firstname TEXT, ci_lastname TEXT);

In [ ]:
import time
i = 0

import sqlite3 as lite

con = lite.connect('/home/wk/myProjects/FinancialModel/data_acquisition/corp.db')
cur = con.cursor()
with con:
    cmdstr = 'select * from corp_addr'
    cur.execute(cmdstr)
    all_record = cur.fetchall()
con.close()

#print(all_record[0])

dba = '/home/wk/myProjects/FinancialModel/data_acquisition/corp.db'
con = lite.connect(dba)
sqlstr = "delete from corp_addr_ext"
con.execute(sqlstr)
con.commit()

for record in all_record:
    i = i + 1
    try:
        ci_corp_id = int(record[0])
        ci_name = str(record[1])
        ci_lab = str(record[2])
        ci_text = str(record[3])
        postal = getPostalCode(ci_text)
        (n, r) = getStrInParentheses(ci_lab) 
    except Exception:
        pass
    
    if i == 1:
        sstr = "insert into corp_addr_ext values "
        
    sstr = sstr + "(" + str(ci_corp_id) + ",'"
    sstr = sstr + ci_name + "','"
    sstr = sstr + ci_lab+ "','"
    sstr = sstr + ci_text + "','"
    sstr = sstr + postal + "','"
    sstr = sstr + " ".join(n[:-1]) + "','"
    sstr = sstr + n[-1:][0] + "'), "
    
    if i > 1000:
        con.execute(sstr[:-2])
        con.commit()
        con.close()
        con = lite.connect(db)
        i = 0
        
con.close()

#### CREATE TABLE addr_cleaned(
#### addr_text TEXT,
#### addr_addr TEXT,
#### addr_box TEXT,
#### addr_unit TEXT,
#### addr_suite TEXT,
#### addr_province TEXT,
#### addr_postal TEXT,
#### addr_country TEXT
#### );

In [ ]:
import sqlite3 as lite
dba = '/home/wk/myProjects/FinancialModel/data_acquisition/corp.db'

con = lite.connect(dba)
cur = con.cursor()
with con:
    cmdstr = 'select ci_text from address order by ci_text DESC'
    cur.execute(cmdstr)
    all_record = cur.fetchall()
con.close()

# Clear the repository
con = lite.connect(dba)
sqlstr = "delete from addr_cleaned"
con.execute(sqlstr)
con.commit()

i = 0
for record in all_record:
    i = i + 1
    new_text = ""
    try:
        ci_text = str(record[0])
        new_text = ci_text.replace("!", "1").replace("É", "E")  
    except Exception:
        pass
    
    (province,postalcode,country,strR) = getProvPSCountry(new_text)
    (unit,strR1) = getUnitNumber("UNIT|Unit|UNIT", strR)
    (suite,strR2A) = getUnitNumber("SUITE|Suite|suite", strR1) 
    (room,strR2B) = getUnitNumber("\sAPP|\sAPT|\sRM|ROOM", strR2A)  
    (box,strR3A) = getUnitNumber("\(?P\.?\s?O\.?\)?\s?BOX\.?|BOX\.?|POB\.X", strR2B)
    (preAddrTxt,strR3B) = getPreText(strR3A.strip())
    strR4= rplceToken("(DRIVE[\s\,])", "DR ", strR3B)
    strR5= rplceToken("(BOULEVARD[\s\,])", "BLVD ", strR4)
    strR6= rplceToken("(CRESCENT[\s\,])", "CRES ", strR5)
    strR7= rplceToken("(STREET[\s\,]|STR[\s\,])", "ST ", strR6)
    strR8= rplceToken("(ROAD[\s\,]|RAOD[\s\,])", "RD ", strR7)
    strR8A= rplceToken("(AVENUE[\s\,])", "AVE ", strR8)
    strR8B= re.sub('[^0-9a-zA-Z`’]+', ' ', strR8A)
    strR8C= rplceToken("(HIGHWAY[\s\,])", "HWY ", strR8B)
    strR8D= rplceToken("(UNIVERSITY[\s\,])", "UVTY ", strR8C)    
    strR9= str.replace(strR8D,".","")
    strR10= rplceToken("\,?(\s+)", " ", strR9).lstrip('-').rstrip()

#    print(strR10 + " | " + province + " | " + postalcode + " | " + country + " | " + unit + " | " + suite + " | " + box + " | ")    
    
    if len(ci_text) > 1:    
        sstr = "insert into addr_cleaned values "        
        sstr = sstr + '("' + ci_text + '","'
        sstr = sstr + strR10.strip() + '","'
        sstr = sstr + preAddrTxt.strip() + '","'
        sstr = sstr + box.strip() + '","'
        sstr = sstr + room.strip() + '","'
        sstr = sstr + unit.strip() + '","'
        sstr = sstr + suite.strip() + '","'
        sstr = sstr + province.strip() + '","'
        sstr = sstr + postalcode.strip() + '","'
        sstr = sstr + country.strip() + '")'
  
        try:
            con.execute(sstr)
            con.commit()
        except Exception:
            pass
    
#    time.sleep(1)
    
    if ((i % 3000) == 1):
        print(str(i) + ": |" + preAddrTxt.strip() + " | " + strR10.strip() + " | " + ci_text)  
    
        
con.close()

In [ ]:
new_text = "SUITE 3500 2 BLOOR STREET WEST TORONTO ON M4W 1A8 CANADA"
(province,postalcode,country,strR) = getProvPSCountry(new_text)
(unit,strR1) = getUnitNumber("UNIT|Unit|UNIT", strR)
(suite,strR2A) = getUnitNumber("SUITE|Suite|suite", strR1) 
(room,strR2B) = getUnitNumber("\sAPT|\sRM|ROOM", strR2A)  
(box,strR3A) = getUnitNumber("\(?P\.?\s?O\.?\)?\s?BOX\.?|BOX\.?|POB\.X", strR2B)
(preAddrTxt,strR3B) = getPreText(strR3A.strip())
strR4= rplceToken("(DRIVE[\s\,])", "DR ", strR3B)
strR5= rplceToken("(BOULEVARD[\s\,])", "BLVD ", strR4)
strR6= rplceToken("(CRESCENT[\s\,])", "CRES ", strR5)
strR7= rplceToken("(STREET[\s\,]|STR[\s\,])", "ST ", strR6)
strR8= rplceToken("(ROAD[\s\,]|RAOD[\s\,])", "RD ", strR7)
strR8A= rplceToken("(AVENUE[\s\,])", "AVE ", strR8)
strR8B= re.sub('[^0-9a-zA-Z`’]+', ' ', strR8A)
strR8C= rplceToken("(HIGHWAY[\s\,])", "HWY ", strR8B)
strR8D= rplceToken("(UNIVERSITY[\s\,])", "UVTY ", strR8C)    
strR9= str.replace(strR8D,".","")
strR10= rplceToken("\,?(\s+)", " ", strR9).lstrip('-').rstrip()
print(strR1)
print(strR2A)
print(strR3A)
print(strR3B)
print(strR4)
print(strR5)

### This is to run address to geocode
### This part is to add more context on the addr: Mainly to get the longitude and latitude of the address

In [ ]:
import sqlite3 as lite

con = lite.connect('/home/wk/myProjects/FinancialModel/data_acquisition/corp.db')
cur = con.cursor()
with con:
    cmdstr = 'select * from addr_cleaned order by addr_text'
    cur.execute(cmdstr)
    all_record = cur.fetchall()
con.close()

import geocoder
import time
url = 'http://localhost/nominatim/'
db = '/home/wk/myProjects/FinancialModel/data_acquisition/corp.db'

def runGeoCoder(buf, url, trial):
    g = None
    t = trial
    if t < 3:
        t = t + 1
        try:
            g = geocoder.osm(buf, url=url)
        except Exception:
            g = runGeoCoder(buf, url, t)
            pass
    return g

icnt = 1
ncnt = 0
for record in all_record:
    ncnt = ncnt + 1
    
    if icnt == 1:cmdstr = 'INSERT INTO addr_complete VALUES '

    buf = record[1]
    buf = buf.replace("UVTY", "UNIVERSITY")
    g = runGeoCoder(buf, url, 1)    

    lon = "0"
    lat = "0"
    raw = ""
    place_id = "0"
    osm_id = "0"
    
    i = 1
    while (g.json is None):
        try:
            buf = buf.split(' ', 1)[1]
            g = runGeoCoder(buf, url, 1)
            sleep(0.1)
        except Exception:
            pass
        i = i + 1
        if i > 2: break
        if len(buf) < 10:break
            
    if  not (g.json is None):
        icnt = icnt + 1
        try:
            lon = g.json["raw"]['lon']
            lat = g.json["raw"]['lat']
        except Exception:
            pass
        
        try:
            state = g.json['raw']['address']['state']
        except Exception:
            pass

        try:
            city = g.json['raw']['address']['city']
        except Exception:
            pass
        
        try:
            country = g.json['raw']['address']['country']
        except Exception:
            pass

        try:    
            raw = g.json['raw']['display_name']
            place_id = g.json["place_id"]
            osm_id = g.json["osm_id"]            
            con = lite.connect(db)
            cur = con.cursor()
            
            cmdstr = cmdstr + "('" + record[0] + "','" + record[1] + "','" + record[2] + "','" + record[3] + "','"
            cmdstr = cmdstr + record[4] + "','" + record[5] + "','" + record[6] + "','" + record[7] + "','"
            cmdstr = cmdstr + record[8] + "','" + record[9] + "'," 
            cmdstr = cmdstr + place_id + "," + osm_id + "," + lon + "," + lat + ',"' + state  + '","' + city + '","' + country + '","' 
            cmdstr = cmdstr + str(raw) + '"), '
            
            if icnt > 1000:
                print (str(ncnt) + ":" + buf + ":" + str(g))
                icnt = 0;
                con = lite.connect(db)
                con.execute(cmdstr[:-2])
                con.commit()
                con.close()            
        except Exception:
            pass
        

In [ ]:
#print(cmdstr[:-2])
print(g.json)
print(icnt)
print(buf)
print(record[1])
print(str(record))
buf = "2 BLOOR STREET WEST TORONTO"
print(runGeoCoder(buf, url, 1).json['raw']['display_name'])
print(runGeoCoder(buf, url, 1).json['raw']['address']['state'])
print(runGeoCoder(buf, url, 1).json['raw']['address']['city'])
print(runGeoCoder(buf, url, 1).json['raw']['address']['country'])

#### This part is to add more context on the addr: Mainly to get the longitude and latitude of the address

In [ ]:
import sqlite3 as lite

con = lite.connect('/home/wk/myProjects/FinancialModel/data_acquisition/corp.db')
cur = con.cursor()
with con:
    cmdstr = 'select ci_text from address where ci_text not in (select ci_text from address_ext where not address = "" ) order by ci_text'
    cur.execute(cmdstr)
    all_record = cur.fetchall()
con.close()


In [ ]:
import geocoder
url = 'http://localhost/nominatim/'
buf = '32 Locheland crescent, ottawa'
geocoder.osm(buf, url=url)

In [ ]:
import time
import geocoder


def getAddressInfo(addrIn):
    addrDictOut = {}
    if len(addrIn) > 0:
        try:
 #           g = geocoder.google(addrIn)
            m = re.search('([^0-9]*)\d+[\,-\]*\s[A-Z]\s+(.+)', addrIn)
            print(m.group(2))
    
            g = geocoder.osm(m.group(2))
            addrDictOut = g.json
            print(addrDictOut)
        except Exception:
            pass
    return (addrDictOut)

def getDictFieldText(d, f):
    retstr = ""
    try:
        retstr = d[f] 
    except Exception:
        pass
    return (retstr)

def getDictFieldValue(d, f):
    retv = 0
    try:
        retstr = d[f] 
    except Exception:
        pass
    return (retv)

In [ ]:
import re
addr = "3 - 4023 26 AVE SW CALGARY AB T3E 0P1 CANADA"
m = re.search('([^0-9]*)\d+[\,-\]*\s[A-Z]\s+(.+)', addr)
print (m.group(2))

In [ ]:
dba = '/home/wk/myProjects/FinancialModel/data_acquisition/corp.db'
con = lite.connect(dba)
sqlstr = "delete from address_ext"
con.execute(sqlstr)
con.commit()

i = 0
for record in all_record:
    i = i + 1
    new_text = ""
    try:
        ci_text = str(record[0])
        new_text = ci_text.replace("!", "1")  
    except Exception:
        pass
    
    d = getAddressInfo(new_text)
    
    e = d
    
    '''
    while len(str(e)) < 10:
        e = getAddressInfo('!0 BAISINGER DRIVE WINNIPEG MB R2N 3Y2 CANADA')
        if len(str(e)) < 10:
            time.sleep(60)
    
    if len(str(d)) < 10:
        d = getAddressInfo(new_text)
    '''
    
    if len(str(d)) < 10:    
        sstr = "insert into address_ext values "        
        sstr = sstr + '("' + ci_text + '","'
        sstr = sstr + getDictFieldText(d, 'address') + '","'
        sstr = sstr + getDictFieldText(d, 'city') + '","'
        sstr = sstr + getDictFieldText(d, 'county') + '","'    
        sstr = sstr + getDictFieldText(d, 'country') + '",'    
        sstr = sstr + str(getDictFieldValue(d, 'lat')) + ','    
        sstr = sstr + str(getDictFieldValue(d, 'lng')) + ',"'
        sstr = sstr + getDictFieldText(d, 'neighborhood') + '","'    
        sstr = sstr + getDictFieldText(d, 'osm_id') + '","'    
        sstr = sstr + getDictFieldText(d, 'postal').replace(" ","") + '","'  
        sstr = sstr + getDictFieldText(d, 'quality') + '",' 
        sstr = sstr + '"' + str(d).replace('"', "^") + '")'    

        try:
            con.execute(sstr)
            con.commit()
        except Exception:
            pass
    
#    time.sleep(1)
    
    if ((i % 1000) == 0):
        print(str(i) + ":" + str(d))
        
con.close()

## Geocoder to Standardize Data

In [ ]:
import geocoder

i = 0
for addr in l_addr:    
    g = geocoder.osm(addr)
    with open("./data/corporate/address.csv","a") as file_w:
        if g.json is None:
            file_w.write(str(i) + "," + ' "' + addr + '",' + '"' + "" + '"' + "\n")
        else:
            file_w.write(str(i) + "," + ' "' + addr + '",' + '"' + str(g.json) + '"' + "\n")
    i+=1
